In [1]:

!jupyter nbconvert --to html --TemplateExporter.exclude_code_cell=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True scc2425-lab6.ipynb 2> /dev/null
!jupyter nbconvert --to slides --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True scc2425-lab6.ipynb 2> /dev/null

# Cloud Computing Systems
## 2024/25

Lab 6
https://smduarte.github.io/scc2425/

Sérgio Duarte, Kevin Gallagher 

# Goals

+ Use Artillery to test a small set of endpoints in the REST API.
+ Create one random test using JavaScript helper functions.
+ Expand the small example to test all of the TuKano endpoints.


# Goals

+ **Use Artillery to test a small set of endpoints in the REST API.**
+ Create one random test using JavaScript helper functions.
+ Expand the small example to test all of the TuKano endpoints.


# Artillery

### What is Artillery?

Artillery is a testing toolkit used to test an application's performance and resilency under load. It works with HTTP APIs, Websocket and Socket.io services, or web applications interacted with via real browsers. It can also integrate well with Azure via Azure Container Instances. In this lesson we will be learning how to do simple tests, as well as extensible tests using Node.js.

# Documentation


### Artillery documentation

[https://www.artillery.io/docs](https://www.artillery.io/docs)

### Specific Features:

#### Basic YAML Scripts
[https://www.artillery.io/docs/reference/test-script](https://www.artillery.io/docs/reference/test-script)

#### Custom Code
[https://www.artillery.io/docs/reference/test-script#processor---load-custom-code](https://www.artillery.io/docs/reference/test-script#processor---load-custom-code)

#### Scenario Weights
[https://github.com/artilleryio/artillery/tree/main/examples/scenario-weights](https://github.com/artilleryio/artillery/tree/main/examples/scenario-weights)

#### Azure Integration (Optional)
[https://www.artillery.io/docs/cicd/azure-devops](https://www.artillery.io/docs/cicd/azure-devops)

# Installing Artillery

## Install npm (if not already installed)

If you don't already have NPM installed, install it by using your machine's package manager. Assuming a Debian-based GNU/Linux, the command is:

```bash
sudo apt install npm
```

## Install Artillery

Execute the command below to install Artillery globally:

```bash
sudo npm install -g artillery@latest
sudo npm install -g faker@latest
sudo npm install -g node-fetch@latest -save
sudo npm install -g https://github.com/preguica/artillery-plugin-metrics-by-endpoint.git
```


# Deploy TuKano

## Make sure to deploy the resources you need.

This can be done by running the management script from lab 5.

`java -cp target/scc2425-mgt-1.0-jar-with-dependencies.jar scc.mgt.AzureManagement`

## TuKano: Local vs Cloud Deployment

For this lab I will assume that you have a local deployment of TuKano. You can deploy TuKano locally using the instructions [here](https://smduarte.github.io/scc2425/labs/proj1/). To change to a cloud deployment, just substitiute **http://127.0.0.1:8080** with your deployment url in the examples below.


# Deploy TuKano

## I needed to do the following to get the TuKano deployment to work:

* Change the TukanoRestServer class to extend a jakarta Application class. This required changes within the constructor. I used Lab1 as a basis for this.
* Create a webapp/WEB-INF folder within the src/main directory of the project. I used Lab1 as a basis for this.
* Modify the web.xml file in the WEB-INF folder to point to the correct web applicaiton class.
* Install necessary dependencies (make sure you use up-to-date versions).
* If you are testing using hibernate because you have not yet implemented cosmos, make sure that you move the hibernate config file to WEB-INF/classes and modify Hibernate.java.


# Preparing for the first test

## Make sure to register the user kgallagher.

Our first test assumes the existence of the user `kgallagher` in the TuKano application. To ensure this user exists, we need to register the user by sending the following JSON payload as a post request to `/rest/users/`.

```json
{
    "userId": "kgallagher",
    "pwd": "easypass",
    "email": "kgallagher@fct.unl.pt",
    "displayName": "kgallagher"
}
```
This can be done via Postman.


# Creating your first get test

After deploying TuKano, you will need to create a simple Artillery test script, which you should store in `first_get.yaml`. Here is an example beginning test.

```yaml
config:
    target: http://127.0.0.1:8080/tukano/rest
    phases:
        name: simple_get
        duration: 1 # how long the test will run for, in seconds
        arrivalRate: 1 # number of arrivals per second.

scenarios:
    - name: TuKanoHome
      flow:
        - get:
            url: /users/kgallagher?pwd=easypass
```


In order for this test to run, you will need to first submit the user kgallagher to the application using the password easypass. You can do this via Postman as described in the last section.

You can run this test using the following command:

`artillery run first_get.yaml`

# Setting up your first post test

We will create our first post test to the TuKano endpoint to register users. To do this we will need a CSV of users that we wish to register. We can use the following CSV, saved as data/users.csv:

```text
userid,pwd,email,displayName
kgallagher2,securepasswordlol,k.gallagher@fct.unl.pt,"Kevin Gallagher"
sduarte,F1n$nVSQ)7js_~w,smd@fct.unl.pt,"Sérgio Duarte"
sDent,riZv02g5@KkK&pq,s.dent@campus.fct.unl.pt,"Stew Dent"
```
We will pass this to artillery and have artillery run this scenario, using data from the CSV as a parameter to the query.


# Post Test Example: config
```yaml
config:                                                                         
    target: http://127.0.0.1:8080/tukano/rest                                          
    phases:                                                                     
        - name: simple_post                                                     
          duration: 1                                                           
          arrivalRate: 3                                                        
    payload:                                                                    
        path: ./data/users.csv                                                  
        order: sequence # selects users sequentially, default is random         
        #loadAll: true # load the whole CSV                                     
        skipHeader: true # The first line is a header, so we want to skip it    
        delimiter: "," # standard delimiter                                     
        skipEmptyLines: true # not important in our case                        
        fields:                                                                 
            - "userId"                                                          
            - "pwd"                                                             
            - "email"                                                           
            - "displayName"
```

# Post Test Example: scenarios
```yaml
scenarios:                                                                      
    - name: TuKanoRegister                                                      
      flow:                                                                     
        - post:                                                                 
            url: /users/                                                        
            headers:                                                            
                Content-Type: application/json                                  
            json:                                                               
                userId: "{{ userId }}" # this parameter is filled from the fields above
                pwd: "{{ pwd }}"                                                
                email: "{{ email }}"                                            
                displayName: "{{ displayName }}"
```
        

# Post Test Output

```
Test run id: tkm3d_tyh9yam8gb6dnfhah4ywmj9b3d66h_rgcp
Phase started: simple_post (index: 0, duration: 1s) 14:47:04(+0100)

Phase completed: simple_post (index: 0, duration: 1s) 14:47:05(+0100)

--------------------------------------
Metrics for period to: 14:47:10(+0100) (width: 1.929s)
--------------------------------------

http.codes.200: ................................................................ 3
http.downloaded_bytes: ......................................................... 23
http.request_rate: ............................................................. 3/sec
http.requests: ................................................................. 3
http.response_time:
  min: ......................................................................... 1246
  max: ......................................................................... 1897
  mean: ........................................................................ 1578.3
  median: ...................................................................... 1587.9
  p95: ......................................................................... 1587.9
  p99: ......................................................................... 1587.9
http.responses: ................................................................ 3
vusers.completed: .............................................................. 3
vusers.created: ................................................................ 3
vusers.created_by_name.TuKanoRegister: ......................................... 3
vusers.failed: ................................................................. 0
vusers.session_length:
  min: ......................................................................... 1261.2
  max: ......................................................................... 1926.7
  mean: ........................................................................ 1595.8
  median: ...................................................................... 1587.9
  p95: ......................................................................... 1587.9
  p99: ......................................................................... 1587.9


All VUs finished. Total time: 3 seconds
```

# Post Test Output

```
--------------------------------
Summary report @ 14:47:08(+0100)
--------------------------------

http.codes.200: ................................................................ 3
http.downloaded_bytes: ......................................................... 23
http.request_rate: ............................................................. 3/sec
http.requests: ................................................................. 3
http.response_time:
  min: ......................................................................... 1246
  max: ......................................................................... 1897
  mean: ........................................................................ 1578.3
  median: ...................................................................... 1587.9
  p95: ......................................................................... 1587.9
  p99: ......................................................................... 1587.9
http.responses: ................................................................ 3
vusers.completed: .............................................................. 3
vusers.created: ................................................................ 3
vusers.created_by_name.TuKanoRegister: ......................................... 3
vusers.failed: ................................................................. 0
vusers.session_length:
  min: ......................................................................... 1261.2
  max: ......................................................................... 1926.7
  mean: ........................................................................ 1595.8
  median: ...................................................................... 1587.9
  p95: ......................................................................... 1587.9
  p99: ......................................................................... 1587.9
```

# Goals

+ Use Artillery to test a small set of endpoints in the REST API.
+ **Create one random test using JavaScript helper functions.**
+ Expand the small example to test all of the TuKano endpoints.


# Randomizing User Details

Now let's imagine we wanted an element of randomness to our test. In order to add randomness, we'll need to create some functionality in NodeJS which we will prompt our Artillery test to run. In this instance we'll start with some methods to create random usernames and passwords.

```JavaScript
function randomUser(char_limit){
    const letters = 'abcdefghijklmnopqrstuvwxyz';
    let username = '';
    let num_chars = Math.floor(Math.random() * char_limit);
    for (let i = 0; i < num_chars); i++) {
        username += letters[Math.floor(Math.random() * letters.length);
    }
    return username;
}
```

# Randomizing User Details

```JavaScript
function randomPassword(pass_len){
    const skip_value = 33;
    const lim_values = 94;

    let password = '';
    let num_chars = Math.floor(Math.random() * pass_len);
    for (let i = 0; i < pass_len; i++) {
        let chosen_char =  Math.floor(Math.random() * lim_values) + skip_value;
        if (chosen_char == "'" || chosen_char == '"')
            i -= 1;
        else
            password += chosen_char
    }
    return password;
}
```

# Randomizing User Details

Now we need a function to tie all of these together.

```JavaScript
function uploadRandomizedUser(requestParams, context, ee, next) {
    let username = randomUsername(10);
    let pword = randomPassword(15);
    let email = username + "@campus.fct.unl.pt";
    let displayName = username;
    
    const user = {
        userId: username,
        pwd: pword,
        email: email,
        displayName: username
    };
    requestParams.body = JSON.stringify(user);
    return next();
} 
```
The parameters to this function are things that will be passed by Artillery. Here we are modifying the request parameters to include the user JSON.

# Randomizing User Details

We will also need a script to handle the server's response. 
```JavaScript
function processRegisterReply(requestParams, response, context, ee, next) {
    if( typeof response.body !== 'undefined' && response.body.length > 0) {
        registeredUsers.push(response.body);
    }
    return next();
} 
```
In this case, registeredUsers is an array of users we have registered to the system.

# Randomizing User Details

Finally, we will set up our test case yaml file to call these functions. 
```yaml
config:                                                                         
  target: http://127.0.0.1:8080/tukano/rest
  plugins:                                                                      
    metrics-by-endpoint: {} # Used to get metrics for each endpoint individually.
  processor: ./test-utils.js
  phases:
  - name: "Warm up"    # WARM UP PHASE: duration: 10s, starting with 1 new client/sec up to 5 new clients/sec
    duration: 10
    arrivalRate: 1
    rampTo: 5
  - name: "Main test"    # MAIN TESTS: duration: 30s, with 10 new clients/sec. Should be varied depending on scenario.
    duration: 30
    arrivalRate: 10
```

# Randomizing User Details


```yaml
scenarios:
  - name: 'Register user'
    weight: 1
    flow:
      - post:
          url: /users                                                         
          headers:
            Content-Type: application/json
            Accept: application/json
          beforeRequest: "uploadRandomizedUser"                                      
          afterResponse: "processRegisterReply" 
```

# Some More Features

You'll notice the following field in the code above:

```yaml
scenarios:
  ...
    weight: 1
  ...
```

Artillery works by assigning *virtual users* (VUs) to a given scenario. If no weight is specified, all scenarios are equally likely to receive a VU. However, we can change the probabilities by assigning weights to scenarios that we wish to happen more frequently. For example, it is more likely that a user will want to view a short than upload one. This could be useful in creating more realistic testing scenarios.

# Some More Features

You'll also notice the following field in the code above:

```yaml
...
plugins:
    metrics-by-endpoint: {}
  ...
```

Artillery provides plugins for us to be able to perform different actions more easily. For example, this line uses a plugin to change the way metrics are reported. By default, metrics are reported in aggregate, meaning the numbers we get are the results across all endpoints. In this case, by using the metrics-by-endpoint plugin we are splitting up and reporting our results per HTTP endpoint, allowing for a more fine-grained understanding of the performance of our application.

# Some More Features

```
--------------------------------------
Metrics for period to: 14:49:50(+0100) (width: 9.005s)
--------------------------------------

http.codes.200: ................................................................ 76
http.codes.409: ................................................................ 15
http.downloaded_bytes: ......................................................... 420
http.request_rate: ............................................................. 10/sec
http.requests: ................................................................. 91
...
http.responses: ................................................................ 91
plugins.metrics-by-endpoint./tukano/rest/users/.codes.200: ..................... 76
plugins.metrics-by-endpoint./tukano/rest/users/.codes.409: ..................... 15
plugins.metrics-by-endpoint.response_time./tukano/rest/users/:
  min: ......................................................................... 2
  max: ......................................................................... 10
  mean: ........................................................................ 4.2
  median: ...................................................................... 4
  p95: ......................................................................... 7.9
  p99: ......................................................................... 8.9
...
```

# Code provided

The code provided [scc2425-lab6-code.zip](scc2425-lab6-code.zip) is a simple set of Artillery tests with the necessary JavaScript functions to run them. 

You will need to ensure that you have TuKano deployed (check slide 7) and have Artillery intsalled (check slide 6). 

To run a test, just run the command

```bash
artillery run <test_yaml_file>
```

After running and ensuring everything works, please extend the tests to include all TuKano endpoints, and weight them to be more realistic to a real world deployment. 

# Goals

+ Use Artillery to test a small set of endpoints in the REST API.
+ Create one random test using JavaScript helper functions.
+ **Expand the small example to test all of the TuKano endpoints.**
